# Deploying ADK Agents to Cloud Run — End-to-End Handbook

This notebook is a **complete, hands-on guide** for deploying agents built with the **Google Agent Development Kit (ADK)** to **Cloud Run**. It covers prerequisites, permissions, API enablement, two deployment paths, testing the API, logs & debugging, **common failure modes and fixes**, and **best practices for secrets**.

## Table of contents
- 1. What you’ll build & how it runs
- 2. Prerequisites
- 3. Install & initialize gcloud CLI
- 4. Project setup: billing, APIs, and IAM
- 5. Agent layout ADK expects
- 6. Path A — `adk deploy cloud_run` (recommended)
- 7. Test the deployed API (UI + curl + SSE)
- 8. Logs & debugging
- 9. Path B — `gcloud run deploy` with your own Dockerfile
- 10. Secrets best practice with Secret Manager
- 11. Common errors and how to fix them
- 12. Clean up & iterate
- 13. My Handles

## 1) What you’ll build & how it runs

A minimal ADK agent (exporting `root_agent`) deployed to **Cloud Run** behind the **ADK API server**. You can call it via REST (`/list-apps`, session endpoints, `/run`, `/run_sse`) or the Dev UI.

```mermaid
flowchart LR
    subgraph Local Dev
        A[Agent code<br/>root_agent in agent.py] --> B[adk deploy cloud_run]
    end

    B --> C[Container build<br/>Cloud Build + Artifact Registry]
    C --> D[Cloud Run Service]
    D <--> E[ADK API Server<br/>/list-apps, /run, /run_sse]
    E --> F[Your Agent]
    F --> G[LLM & Tools e.g., MCP]
    
    style A fill:#e1f5fe
    style B fill:#e1f5fe
    style C fill:#f3e5f5
    style D fill:#e8f5e8
    style E fill:#fff3e0
    style F fill:#e8f5e8
    style G fill:#ffebee
```

## 2) Prerequisites

- A Google Cloud project with **billing enabled**.
- One of the two **auth modes**:
  - **Vertex AI path** (recommended on GCP): `GOOGLE_CLOUD_PROJECT`, `GOOGLE_CLOUD_LOCATION`, `GOOGLE_GENAI_USE_VERTEXAI=True`
  - **AI Studio key path**: `GOOGLE_CLOUD_PROJECT`, `GOOGLE_CLOUD_LOCATION`, `GOOGLE_GENAI_USE_VERTEXAI=False`, `GOOGLE_API_KEY`
- Python 3.9+ and `pip`.
- **ADK**: `pip install google-adk`
- (Optional) `jq` for pretty-printing JSON in shell output.

## 3) Install & initialize gcloud CLI

**macOS (Homebrew)**
```bash
brew install google-cloud-sdk
```

**Debian/Ubuntu**
```bash
sudo apt-get update && sudo apt-get install -y apt-transport-https ca-certificates gnupg
echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt cloud-sdk main" | sudo tee /etc/apt/sources.list.d/google-cloud-sdk.list
curl -fsSL https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo gpg --dearmor -o /usr/share/keyrings/google-cloud.gpg
sudo apt-get update && sudo apt-get install -y google-cloud-cli
```

**Windows**
Download installer from Google Cloud or `choco install gcloudsdk`.

**Initialize & auth**
```bash
gcloud auth login
gcloud config set project <YOUR_PROJECT_ID>
gcloud config get-value account
gcloud config get-value project
```

## 4) Project setup: billing, APIs, and IAM

### 4.1 Billing
Ensure billing is linked to the project (Console → Billing).

### 4.2 Enable required APIs
```bash
PROJECT=<YOUR_PROJECT_ID>
gcloud services enable   run.googleapis.com   artifactregistry.googleapis.com   cloudbuild.googleapis.com   logging.googleapis.com   secretmanager.googleapis.com   --project $PROJECT

gcloud services list --enabled --project $PROJECT | egrep 'run|artifact|cloudbuild|logging|secret'
```

### 4.3 IAM must-haves (for the **user** doing the deploy)
If you are **Project Owner**, you’re covered. Otherwise, grant:
- `roles/run.admin` (or `run.developer`)
- `roles/iam.serviceAccountUser`
- `roles/artifactregistry.writer` (or `.admin`)
- `roles/cloudbuild.builds.editor`
- `roles/serviceusage.serviceUsageConsumer`

Check your roles:
```bash
gcloud projects get-iam-policy $PROJECT   --flatten="bindings[].members"   --filter="bindings.members:user:$(gcloud config get-value account)"   --format="table(bindings.role)"
```

## 5) Agent layout ADK expects

```
your-agent/
└─ capital_agent/
   ├─ __init__.py        # contains: from . import agent
   ├─ agent.py           # exports: root_agent
   └─ requirements.txt   # agent-specific extras (ADK server is bundled by deployer)
```

**`agent.py` example**
```python
from google.adk.agents.llm_agent import Agent

root_agent = Agent(
    model="gemini-2.0-flash",
    name="capital_agent",
    instruction=(
        "Answer concisely. If asked for a country's capital, reply with only the capital and a one-line confirmation."
    ),
)
```

> Cloud Run **service names must be dash-case** (e.g., `capital-service`).

## 6) Path A — `adk deploy cloud_run` (recommended)

**Choose ONE auth path**

**Vertex AI**
```bash
export GOOGLE_CLOUD_PROJECT="<your-project-id>"
export GOOGLE_CLOUD_LOCATION="us-central1"
export GOOGLE_GENAI_USE_VERTEXAI=True
```

**AI Studio key**
```bash
export GOOGLE_CLOUD_PROJECT="<your-project-id>"
export GOOGLE_CLOUD_LOCATION="us-central1"
export GOOGLE_GENAI_USE_VERTEXAI=False
export GOOGLE_API_KEY="<your-ai-studio-api-key>"
```

**Optional names**
```bash
export SERVICE_NAME="capital-service"
export APP_NAME="capital_agent"
export AGENT_PATH="./capital_agent"
```

**Deploy**
```bash
adk deploy cloud_run   --project=$GOOGLE_CLOUD_PROJECT   --region=$GOOGLE_CLOUD_LOCATION   --service_name=$SERVICE_NAME   --app_name=$APP_NAME   --with_ui   $AGENT_PATH
```

## 8) Logs & debugging

Cloud Console → **Cloud Run** → your service → **Logs**. You’ll see ADK runtime logs (model calls, tool calls, errors).

```mermaid
flowchart LR
  A[Client] -->|/run| B[Cloud Run Service]
  B --> C[ADK API Server]
  C --> D[Agent Execution]
  D --> E[Logs: LLM calls, tools, state deltas]
```

## 9) Path B — `gcloud run deploy` with your own Dockerfile

Use when you need custom system packages or controlled runtime.

**Dockerfile sketch**
```dockerfile
FROM python:3.11-slim
RUN apt-get update && apt-get install -y --no-install-recommends git curl && rm -rf /var/lib/apt/lists/*
WORKDIR /app
COPY requirements.txt /app/requirements.txt
RUN pip install --no-cache-dir -r /app/requirements.txt
COPY agents/ /app/agents/
ENV PORT=8000
EXPOSE 8000
```
Deploy:
```bash
gcloud run deploy <service-name> --source .   --project $GOOGLE_CLOUD_PROJECT --region $GOOGLE_CLOUD_LOCATION   --allow-unauthenticated
```

## 11) Common errors and how to fix them

| Error/Symptom | Cause | Fix |
|---|---|---|
| `PERMISSION_DENIED: permission to enable service [run.googleapis.com]` | Missing **Service Usage** perms | Ask owner for `roles/serviceusage.serviceUsageAdmin` (enable) and `roles/serviceusage.serviceUsageConsumer` (use) |
| `Cannot access API run.googleapis.com` after enabling | You don’t have **Run** or **Service Usage** roles | Grant `roles/run.admin`/`run.developer` and `roles/serviceusage.serviceUsageConsumer` |
| `No root_agent found` | Wrong package layout | Ensure `agent.py` exports `root_agent` and `__init__.py` has `from . import agent` |
| Service name rejected | Underscores in name | Use dash-case (`capital-service`) |
| 401 calling API | Private service | Use identity token: `gcloud auth print-identity-token` |
| 500 on `/run` | Env combo mismatch or missing keys | Check `GOOGLE_GENAI_USE_VERTEXAI` True/False pairing; set `GOOGLE_API_KEY` if needed; check logs |
| MCP fails in container | Missing tools (`uv`, `git`) | Use Path B with a Dockerfile installing required packages |

## 13) My Handles

Linkedin :- https://www.linkedin.com/in/mayank953/  
Youtube :- https://www.youtube.com/@tech.mayankagg  
Instagram :- https://www.instagram.com/tech.mayankagg/  
Substack :- https://aiwithmayank.substack.com  
Medium :- https://medium.com/@tech.mayankagg  
Udemy :- https://www.udemy.com/user/mayank-aggarwal-197/  
Github :- https://github.com/mayank953/